In [3]:
import os
if os.getcwd() == '/home/user/code':
    os.chdir('/home/user/code/nlp2024_ClefTask4SOTA')

%load_ext autoreload
%autoreload 2

# What should be the max length of the model?

Approach: Tokenize the longest annotation in the dataset, this is the max lenght.

In [8]:
from src.dataset import TDMSDataset, PATH


# load all annotations:
dataset = TDMSDataset(PATH.TRAIN)

annotations = []
for i in range(len(dataset)):
    f, tex, jsn = dataset[i]
    annotations.append(str(jsn))


In [23]:
import pandas as pd


df = pd.DataFrame(annotations)
df.columns = ["annotation"]
df

,annotation
0,[{'LEADERBOARD': {'Task': 'Thoracic Disease Cl...
1,[{'LEADERBOARD': {'Task': 'Hate Speech Detecti...
2,unanswerable\n
3,unanswerable\n
4,unanswerable\n
...,...
12283,unanswerable\n
12284,[{'LEADERBOARD': {'Task': 'Semi-Supervised Vid...
12285,[{'LEADERBOARD': {'Task': 'Video Quality Asses...
12286,[{'LEADERBOARD': {'Task': 'Skeleton Based Acti...


In [30]:
df["str_len"] = df.apply(lambda row: len(row["annotation"]), axis=1)

# top 10 longest annotations by string lenght:
df_t10 = df.sort_values(by="str_len", ascending=False)[:10]
df_t10

,annotation,str_len
4701,[{'LEADERBOARD': {'Task': 'Node Classification...,56561
5067,[{'LEADERBOARD': {'Task': 'Video Object Segmen...,25627
3174,[{'LEADERBOARD': {'Task': 'Visual Question Ans...,24637
7672,[{'LEADERBOARD': {'Task': 'Semi-Supervised Vid...,24078
10566,[{'LEADERBOARD': {'Task': 'Semi-Supervised Vid...,23441
5173,"[{'LEADERBOARD': {'Task': 'Atari Games', 'Data...",22760
11829,[{'LEADERBOARD': {'Task': 'Implicit Relations'...,19365
3346,[{'LEADERBOARD': {'Task': 'image-sentence alig...,17769
11865,"[{'LEADERBOARD': {'Task': 'Atari Games', 'Data...",16787
10021,[{'LEADERBOARD': {'Task': 'Automatic Lyrics Tr...,16043


In [57]:
# ignoring the outlier of 56561 chars and all below 20k chars as candidates for longest annotation

df_longest = df_t10[1:6]
df_longest


,annotation,str_len
5067,[{'LEADERBOARD': {'Task': 'Video Object Segmen...,25627
3174,[{'LEADERBOARD': {'Task': 'Visual Question Ans...,24637
7672,[{'LEADERBOARD': {'Task': 'Semi-Supervised Vid...,24078
10566,[{'LEADERBOARD': {'Task': 'Semi-Supervised Vid...,23441
5173,"[{'LEADERBOARD': {'Task': 'Atari Games', 'Data...",22760


In [58]:
# is the longest annotation character wise also the longest annotation token wise?
from transformers import AutoModelForCausalLM, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.3")

df_longest["token_len"] = df_longest.apply(lambda row: len(tokenizer(row["annotation"])["input_ids"]), axis=1)
df_longest["token_len"].max()

# longest = 9878 tokens => Max len should be 10k tokens

/tmp/ipykernel_101300/2527748780.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_longest["token_len"] = df_longest.apply(lambda row: len(tokenizer(row["annotation"])["input_ids"]), axis=1)


9878

In [54]:
# Sanity check: Other Tokenizers (llama)

from transformers import AutoModelForCausalLM, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("meta-llama/Meta-Llama-3-8B")

df_longest["token_len"] = df_longest.apply(lambda row: len(tokenizer(row["annotation"])["input_ids"]), axis=1)
df_longest["token_len"].max()

# longest = 8670 tokens => Max len should be 8700 tokens

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/tmp/ipykernel_101300/974574506.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_longest["token_len"] = df_longest.apply(lambda row: len(tokenizer(row["annotation"])["input_ids"]), axis=1)


8670

In [59]:
# Sanity check: Other Tokenizers (gemma)

from transformers import AutoModelForCausalLM, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("google/gemma-7b")

df_longest["token_len"] = df_longest.apply(lambda row: len(tokenizer(row["annotation"])["input_ids"]), axis=1)
df_longest["token_len"].max()

# longest = 8898 tokens => Max len should be 9k tokens

/tmp/ipykernel_101300/3947468084.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_longest["token_len"] = df_longest.apply(lambda row: len(tokenizer(row["annotation"])["input_ids"]), axis=1)


8898

# Model Experiments

In [ ]:
# Llama 3 8b vs Mistral 7b vs Gemma 7b

In [4]:
from src.models import HFModel


mistral_it = HFModel("mistralai/Mistral-7B-Instruct-v0.3", 0)
gemma_it = HFModel("google/gemma-7b-it", 1)
# llama_it = HFModel("meta-llama/Meta-Llama-3-8B-Instruct", 2)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

You set `add_prefix_space`. The tokenizer needs to be converted from the slow tokenizers
You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
`config.hidden_act` is ignored, you should use `config.hidden_activation` instead.
Gemma's activation function will be set to `gelu_pytorch_tanh`. Please, use
`config.hidden_activation` if you want to override this behaviour.
See https://github.com/huggingface/transformers/pull/29402 for more details.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [7]:
llama_it.generate("Give an example of a kind of fish, reply be stating the name only.")

KeyboardInterrupt: 

In [8]:
from src.content_extraction import naive_doctaet, parse_response



CONTEXT_LEN = 8192

def extract_doctaet(model, prompt_template, tex):
    doctaet = naive_doctaet(tex)
    cut_doctaet = model.cut_text(doctaet, CONTEXT_LEN - model.num_tokens(prompt_template("")))
    prompt = prompt_template(cut_doctaet)
    response = model.generate(prompt)
    parsed =  parse_response(response)
    return parsed

In [9]:
# extract_doctaet(mistral_it, zero_shot_template_initial, tex)

NameError: name 'tex' is not defined

In [ ]:
from src.experiment_runner import Experiment, run
from src.prompt_templates import zero_shot_template_initial, few_shot_template_initial
exps = [
    Experiment(mistral_it, zero_shot_template_initial, extract_doctaet, "mistral-it_zs_initial"),
    Experiment(mistral_it, few_shot_template_initial, extract_doctaet, "mistral-it_fs_initial"),
    Experiment(gemma_it, zero_shot_template_initial, extract_doctaet, "gemma-it_zs_initial"),
    Experiment(gemma_it, few_shot_template_initial, extract_doctaet, "gemma-it_fs_initial"),
]

dfs = []
for exp in exps:
    df = run(exp, PATH.VAL)
    dfs.append(df)

for exp in exps:
    df = run(exp, PATH.TEST)
    dfs.append(df)

In [1]:
import torch
torch.cuda.empty_cache()